In [1]:
import pandas as pd
import json

In [2]:
# ---------------------------
# 0) File paths (edit if needed)
# ---------------------------
orders_fp = "order_data_cleaned_and_encoded.csv"    # your cleaned orders
cust_fp   = "customer_data_cleaned_and_encoded.csv"             # your encoded customers (registered/guest/special_membership)
store_fp  = "store_data_cleaned_and_encoded.csv"                # your encoded stores (store_number + city dummies + STATE + flags)
out_fp    = "master_orders_customers_stores.csv"

In [3]:
# ---------------------------
# 1) Read files and normalize key dtypes
# ---------------------------
orders = pd.read_csv(orders_fp, dtype=str)   # read everything as str to avoid join-dtype problems
cust   = pd.read_csv(cust_fp, dtype=str)
store  = pd.read_csv(store_fp, dtype=str)

# If some numeric columns are needed later, you'll cast them back.
# Ensure keys exist
assert 'CUSTOMER_ID' in orders.columns, "orders missing CUSTOMER_ID"
assert 'STORE_NUMBER' in orders.columns, "orders missing STORE_NUMBER"
assert 'CUSTOMER_ID' in cust.columns,   "customer file missing CUSTOMER_ID"
assert 'STORE_NUMBER' in store.columns,  "store file missing STORE_NUMBER"

# Keep a copy of the original row count for verification
n_orders = len(orders)

In [4]:
# ---------------------------
# 2) Prepare customer frame: drop any original CUSTOMER_TYPE (if present) & prefix columns
# ---------------------------
# Drop raw CUSTOMER_TYPE if present (we only want the one-hot columns)
if 'CUSTOMER_TYPE' in cust.columns:
    cust = cust.drop(columns=['CUSTOMER_TYPE'])

# Identify customer feature columns (all except CUSTOMER_ID)
cust_feat_cols = [c for c in cust.columns if c != 'CUSTOMER_ID']

# Prefix them to avoid collisions after merge
cust_prefixed = cust.rename(columns={c: f"cust_{c}" for c in cust_feat_cols})
# keep the key
cust_prefixed = cust_prefixed[['CUSTOMER_ID'] + [f"cust_{c}" for c in cust_feat_cols]]

In [5]:
# ---------------------------
# 3) Prepare store frame: prefix store columns (except STORE_NUMBER)
# ---------------------------
store_feat_cols = [c for c in store.columns if c != 'STORE_NUMBER']
store_prefixed = store.rename(columns={c: f"store_{c}" for c in store_feat_cols})
store_prefixed = store_prefixed[['STORE_NUMBER'] + [f"store_{c}" for c in store_feat_cols]]


In [6]:
# ---------------------------
# 4) Left join orders <- customers
# ---------------------------
merged = orders.merge(cust_prefixed, on='CUSTOMER_ID', how='left', validate='m:1')

print(f"Orders: {n_orders} rows; after customer-join: {len(merged)} rows")

# Count how many orders had no matching customer row (before we fill defaults)
cust_missing_count = merged[[f"cust_{c}" for c in cust_feat_cols]].isna().all(axis=1).sum()
print(f"Orders with NO matching customer record: {cust_missing_count}")

Orders: 1414410 rows; after customer-join: 1414410 rows
Orders with NO matching customer record: 0


In [7]:
# ---------------------------
# 5) Left join (orders+cust) <- stores
# ---------------------------
merged = merged.merge(store_prefixed, on='STORE_NUMBER', how='left', validate='m:1')
print(f"After store-join: {len(merged)} rows")

# Count how many orders have no matching store info
# We'll look for missing STATE in store (store_STATE is expected); otherwise fall back to any store column
store_state_col = next((c for c in merged.columns if c.lower().endswith('state') and c.startswith('store_')), None)
if store_state_col:
    store_missing_count = merged[store_state_col].isna().sum()
else:
    # fallback — check any store_ column
    store_cols = [c for c in merged.columns if c.startswith('store_')]
    store_missing_count = merged[store_cols].isna().all(axis=1).sum() if store_cols else 0

print(f"Orders with NO matching store record: {store_missing_count}")

After store-join: 1414410 rows
Orders with NO matching store record: 48223


In [8]:
# ---------------------------
# 6) Fill defaults for customer one-hot columns
#    - registered, guest (0/1) -> fill 0 when unknown
#    - special_membership -> fill 1 when the original CUSTOMER_TYPE was NaN or customer absent
# ---------------------------
# Determine cust column names (after prefix)
cust_cols_prefixed = [c for c in merged.columns if c.startswith('cust_')]

# If the exact names exist (expected from earlier step) use them; otherwise just handle all cust_ columns
# Create a boolean mask for rows that have no customer info (all cust_ cols are NA)
if cust_cols_prefixed:
    missing_cust_mask = merged[cust_cols_prefixed].isna().all(axis=1)
    # Fill registered/guest with 0 where missing; fill special with 1 where missing
    # If you have specific named columns, handle explicitly:
    if 'cust_registered' in merged.columns and 'cust_guest' in merged.columns and 'cust_special_membership' in merged.columns:
        merged['cust_registered'] = merged['cust_registered'].fillna(0).astype(int)
        merged['cust_guest'] = merged['cust_guest'].fillna(0).astype(int)
        # temporarily fill special with 0, then set to 1 where mask==True
        merged['cust_special_membership'] = merged['cust_special_membership'].fillna(0).astype(int)
        merged.loc[missing_cust_mask, 'cust_special_membership'] = 1
    else:
        # Generic fallback: fill all cust_ NA -> 0 (safe) and set a synthetic flag if all were NA
        merged[cust_cols_prefixed] = merged[cust_cols_prefixed].fillna(0)
        # create an indicator
        merged['cust_info_missing'] = missing_cust_mask.astype(int)
else:
    print("No customer feature columns found with prefix 'cust_' — skipping cust fill step.")

In [9]:
# ---------------------------
# 7) Fill defaults for store columns
#    - city dummies -> fill 0
#    - store_STATE -> fill 'Unknown'
#    - store_city_missing / store_state_missing -> fill 1 where missing store
# ---------------------------
store_cols_prefixed = [c for c in merged.columns if c.startswith('store_')]

# Identify city dummy columns (common prefix used earlier was 'city_' inside store, so now becomes 'store_city_')
store_city_cols = [c for c in store_cols_prefixed if c.startswith('store_city_')]

# Fill city dummies with 0
if store_city_cols:
    merged[store_city_cols] = merged[store_city_cols].fillna(0).astype(int)

# Fill state
if store_state_col:
    merged[store_state_col] = merged[store_state_col].fillna('Unknown')
else:
    # nothing to fill specifically, but ensure other store cols are not NaN
    merged[store_cols_prefixed] = merged[store_cols_prefixed].fillna(0)

# Fill known store-missing indicator flags if they exist (store_city_missing, store_state_missing)
for flag in ['store_city_missing', 'store_state_missing']:
    if flag in merged.columns:
        # if merge resulted in NA (no store row), mark missing (1)
        merged[flag] = merged[flag].fillna(1).astype(int)

In [10]:
# ---------------------------
# 8) Sanity checks
# ---------------------------
#  - number of rows should equal original orders row count
assert len(merged) == n_orders, "Row count changed — check join keys / duplicates"

print("Final master shape:", merged.shape)
print("Columns sample:", merged.columns.tolist()[:40])

# Optional: count of distinct customers/stores matched
matched_customers = merged['cust_registered'].notna().sum() if 'cust_registered' in merged.columns else None
print("Sample counts -> matched_customers (non-null registered flag):", matched_customers)

#

Final master shape: (1414410, 175)
Columns sample: ['CUSTOMER_ID', 'STORE_NUMBER', 'ORDER_CREATED_DATE', 'ORDER_ID', 'ORDER_CHANNEL_NAME', 'ORDER_SUBCHANNEL_NAME', 'ORDER_OCCASION_NAME', '$19.99 Crispy Feast', '10 pc Grilled Wings', '10 pc Grilled Wings Combo', '10 pc Mixed Wings', '10 pc Mixed Wings Combo', '10 pc Spicy Wings', '10 pc Spicy Wings Combo', '100 pc Family Grilled Wings', '100 pc Family Mixed Wings', '100 pc Family Spicy Wings', '100 pc Grilled Wings', '100 pc Mixed Wings', '100 pc Spicy Wings', '15 pc Crispy Strips', '15 pc Grilled Wings', '15 pc Grilled Wings Combo', '15 pc Mixed Wings', '15 pc Mixed Wings Combo', '15 pc Spicy Wings', '15 pc Spicy Wings Combo', '2 pc Crispy Strips', '20 Oz Soda', '20 pc Crispy Strips', '20 pc Grilled Wings', '20 pc Mixed Wings', '20 pc Spicy Wings', '20pc Spicy Feast Deal', '24 pc Family Grilled Wings', '24 pc Family Mixed Wings', '24 pc Family Spicy Wings', '25 pc Game Day Pack', '3 Strips Lunch', '3 pc Crispy Strips Combo']
Sample cou

In [11]:
# ---------------------------
# 9) Save final master CSV
# ---------------------------
merged.to_csv(out_fp, index=False)
print("Saved master dataset to:", out_fp)

Saved master dataset to: master_orders_customers_stores.csv


In [12]:
final_sample = merged.head(50)

In [13]:
final_sample.to_csv("final_sample.csv", index=False)